In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler


from PIL import Image
import cv2

# Deep learning feature extraction
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model


In [ ]:
# Paths to input data
data_path = "/kaggle/input/csiro-biomass"
train_df = pd.read_csv(f"{data_path}/train.csv")
test_df = pd.read_csv(f"{data_path}/test.csv")
sample_sub = pd.read_csv(f"{data_path}/sample_submission.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
train_df.head()


In [ ]:
train_dir = f"{data_path}/train"
test_dir = f"{data_path}/test"

print("Number of train images:", len(os.listdir(train_dir)))
print("Number of test images:", len(os.listdir(test_dir)))

import matplotlib.pyplot as plt
import random

sample_images = random.sample(os.listdir(train_dir), 3)
plt.figure(figsize=(12, 4))
for i, img_name in enumerate(sample_images):
    img = Image.open(os.path.join(train_dir, img_name))
    plt.subplot(1, 3, i+1)
    plt.imshow(img)
    plt.title(img_name)
    plt.axis('off')
plt.show()


In [ ]:
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model


base_model = ResNet50(weights=None, include_top=False, pooling='avg')


model = Model(inputs=base_model.input, outputs=base_model.output)

print("✅ ResNet50 model created successfully (no pretrained weights).")

def extract_features(image_folder, image_paths):
    features = []
    for img_path in tqdm(image_paths):
        # Support full paths and base names
        if not os.path.exists(img_path):
            img_path = os.path.join(image_folder, os.path.basename(img_path))
        if os.path.exists(img_path):
            img_data = load_and_preprocess_image(img_path)
            feat = model.predict(img_data, verbose=0)
            features.append(feat.flatten())
        else:
            features.append(np.zeros(2048))  # fallback
    return np.array(features)


In [ ]:
print("Train columns:", train_df.columns.tolist())
print("Test columns:", test_df.columns.tolist())
train_df.head()


In [ ]:
train_features = extract_features(train_dir, train_df['image_path'])
test_features = extract_features(test_dir, test_df['image_path'])


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler


target_cols = train_df['target_name'].unique().tolist()
print("Targets found:", target_cols)


scaler = StandardScaler()
X = scaler.fit_transform(train_features)
X_test = scaler.transform(test_features)


In [ ]:

target_cols = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']

scaler = StandardScaler()
X = scaler.fit_transform(train_features)
X_test = scaler.transform(test_features)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm

models = {}
preds = []

# List of targets
target_cols = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']

for target in target_cols:
    print(f"Training model for {target}...")

   
    target_df = train_df[train_df['target_name'] == target].reset_index(drop=True)

  
    image_ids = target_df['image_path'].apply(lambda x: x.split('/')[-1].replace('.jpg', ''))

  
    X_target = []
    for img_id in image_ids:
        idx = train_df['image_path'].apply(lambda x: x.split('/')[-1].replace('.jpg', '')).tolist().index(img_id)
        X_target.append(train_features[idx])
    X_target = np.array(X_target)


    y = target_df['target'].values

    
    model = RandomForestRegressor(n_estimators=200, random_state=42)
    model.fit(X_target, y)
    models[target] = model

  
    preds.append(model.predict(X_test))


preds = np.column_stack(preds)  
print("✅ Prediction shape:", preds.shape)


In [ ]:
# ✅ Ensure preds and test_df match
num_preds = len(preds)
num_test = len(test_df)

print(f"🔍 Predictions: {num_preds}, Test Images: {num_test}")

if num_preds != num_test:
    print("⚠️ Warning: Mismatch detected! Adjusting...")
    if num_preds > num_test:
        preds = preds[:num_test]
    else:
        avg_pred = np.mean(preds, axis=0)
        while len(preds) < num_test:
            preds = np.vstack([preds, avg_pred])

# ✅ Create submission
submission = []

# Extract image IDs without file extensions
test_image_ids = test_df['image_path'].apply(lambda x: x.split('/')[-1].replace('.jpg', '')).values

for i, img_id in enumerate(test_image_ids):
    for j, target in enumerate(target_cols):
        sample_id = f"{img_id}__{target}"
        submission.append([sample_id, preds[i][j]])

# ✅ Convert to DataFrame and save
submission_df = pd.DataFrame(submission, columns=['sample_id', 'target'])
submission_df.to_csv('/kaggle/working/submission.csv', index=False)

print("✅ submission.csv created successfully!")
print(f"📄 Total rows in submission: {len(submission_df)}")

# Show first few rows for sanity check
submission_df.head(10)
